In [1]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig,GPT2Tokenizer
from transformers import GPT2Tokenizer,GPT2Model,AutoModel
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import LineByLineTextDataset
from tokenizers import Tokenizer
from datasets import load_dataset

2024-10-18 18:21:56.475476: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-18 18:21:56.489046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-18 18:21:56.504266: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-18 18:21:56.560038: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-18 18:21:56.572464: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import os

# 设置环境变量
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

# 打印环境变量以确认设置成功
print(os.environ.get('HF_ENDPOINT'))

https://hf-mirror.com


In [3]:
tokenizer = GPT2Tokenizer.from_pretrained("dnagpt/dna_eng_bpe")


In [4]:
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=1024,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config) #for pre train

In [8]:
# 1. load dna +english dataset
raw_dataset = load_dataset('dnagpt/dna_english_seq')
dataset = raw_dataset["train"].train_test_split(test_size=0.1, shuffle=True)

# 2. tokenize
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=1024)

# 3. 对数据集应用分词函数
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

# 4. 创建一个数据收集器，用于动态填充和遮蔽
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)

Generating train split:   0%|          | 0/482125 [00:00<?, ? examples/s]

Map:   0%|          | 0/433912 [00:00<?, ? examples/s]

Map:   0%|          | 0/48213 [00:00<?, ? examples/s]

In [9]:
out_model_path = "gpt_dna_eng_v1"
train_epoches = 3
batch_size = 20


training_args = TrainingArguments(
        output_dir=out_model_path,
        overwrite_output_dir=True,
        num_train_epochs=train_epoches,
        per_device_train_batch_size=batch_size,
        save_steps=2000,
        save_total_limit=2,
        prediction_loss_only=True,
        #fp16=True, v100没法用
    )


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
)

Detected kernel version 4.19.90, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [10]:
trainer.train()
trainer.save_model(out_model_path)

Step,Training Loss
500,5.639100
1000,4.992700
1500,4.811000
2000,4.812500
2500,4.709200
3000,4.587900
3500,4.502900
4000,4.459300
4500,4.405000
5000,4.358900


In [11]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 37.43


In [12]:
model.push_to_hub("gpt2_dna_eng_v1", organization="dnagpt", use_auth_token="hf_**********")

/root/miniconda3/lib/python3.12/site-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/transformers/utils/hub.py:731: UserWarning: The `organization` argument is deprecated and will be removed in v5 of Transformers. Set your organization directly in the `repo_id` passed instead (`repo_id={organization}/{model_id}`).
  warnings.warn(


ConnectTimeout: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/repos/create (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fac91ef9f10>, 'Connection to huggingface.co timed out. (connect timeout=None)'))"), '(Request ID: 37c06293-9199-4352-9063-47247684bf71)')